In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
gbif_r = pd.read_csv('gbif_r.csv')

In [3]:
# New dataframe with the lat-long mean and median per year and specie
dif = pd.DataFrame(gbif_r.groupby(['year', 'species'])['lat'].mean())
dif = dif.rename(columns = {'lat': 'lat_mean'})
dif['long_mean'] = gbif_r.groupby(['year', 'species'])['long'].mean()
dif['long_med'] = gbif_r.groupby(['year', 'species'])['long'].median()
dif['lat_med'] = gbif_r.groupby(['year', 'species'])['lat'].median()
dif['regions'] = gbif_r.groupby(['year', 'species'])['region'].agg(lambda x:x.value_counts().index[0])

dif.reset_index(inplace = True)  

dif

,year,species,lat_mean,long_mean,long_med,lat_med,regions
0,2001,Accipiter gentilis,41.475405,-3.764305,-4.621390,41.910837,Castilla y León
1,2001,Accipiter nisus,40.340947,-4.042108,-4.867379,41.369110,Andalucía
2,2001,Acrocephalus arundinaceus,41.010709,-3.688138,-4.752939,42.067673,Castilla y León
3,2001,Acrocephalus melanopogon,40.521891,2.150216,3.105440,40.540875,Cataluña
4,2001,Acrocephalus paludicola,41.894325,-4.745975,-4.752939,42.067673,Castilla y León
...,...,...,...,...,...,...,...
8765,2020,Uria aalge,43.424461,-3.819587,-3.462010,43.420030,País Vasco
8766,2020,Vanellus gregarius,39.886925,-4.772777,-5.954113,39.560776,Extremadura
8767,2020,Vanellus vanellus,40.332819,-3.799163,-4.752939,40.173980,Extremadura
8768,2020,Xema sabini,43.200982,-6.730901,-7.677165,43.414082,Galicia


In [ ]:
plt.figure(figsize = (13,3))
sns.kdeplot(x = gbif_r.year, color = 'r');

In [ ]:
plt.figure(figsize = (15,12))
plot_order = sns.histplot(y = gbif_r.region, 
                          hue = gbif_r.order,
                           multiple = "fill",
                         palette = 'Paired');

plot_order.set_title('Bird distribution by order and regions')
plot_order.set(xlabel = 'Order', 
                ylabel='Region');

a = gbif_r.groupby(['observations', 'year'], as_index = False)['species']

dfa = pd.DataFrame(data = a)
dfa

b = gbif_r.groupby(['species','year'], as_index = False)['lat'].sum()
dfb = pd.DataFrame(b)
dfb

# 
years_list = gbif_r.year.unique()
def funcion(years_list):
    for y in years_list:
        if gbif_r.year == y:
            mean1 = gbif_r.groupby(['species'])['lat'].mean
        else: 
            print('there is an error')
    return mean1

y20 = gbif_r[gbif_r['year'] == 2020]
y19 = gbif_r[gbif_r['year'] == 2019]
y18 = gbif_r[gbif_r['year'] == 2018]
y17 = gbif_r[gbif_r['year'] == 2017]
y16 = gbif_r[gbif_r['year'] == 2016]
y15 = gbif_r[gbif_r['year'] == 2015]
y14 = gbif_r[gbif_r['year'] == 2014]
y13 = gbif_r[gbif_r['year'] == 2013]
y12 = gbif_r[gbif_r['year'] == 2012]
y11 = gbif_r[gbif_r['year'] == 2011]
y10 = gbif_r[gbif_r['year'] == 2010]
y09 = gbif_r[gbif_r['year'] == 2009]
y08 = gbif_r[gbif_r['year'] == 2008]
y07 = gbif_r[gbif_r['year'] == 2007]
y06 = gbif_r[gbif_r['year'] == 2006]
y05 = gbif_r[gbif_r['year'] == 2005]
y04 = gbif_r[gbif_r['year'] == 2004]
y03 = gbif_r[gbif_r['year'] == 2003]
y02 = gbif_r[gbif_r['year'] == 2002]
y01 = gbif_r[gbif_r['year'] == 2001]

year_list = [y20, y19, y18, y17, y16, y15, y14, y13, y12, y11, y10, y09, y08, y07, y06, y05, y04, y03, y02, y01]


# for y in year_list
#   y['latlong'] = y['lat'].astype(str) + ', ' + y['long'].astype(str)

funcion que compare de cada especie-año la long /lat ... y me devuelva una lista de
aquellas especies que tengan una diferencia de valor mayor a X 
o que me ordene la diferencia de valor encontrada, y yo hacer un sort y trabajar con las mayores diferencias


# nombrecolumna = list(dif.columns)
species = list(dif.species.unique()) 
def compare(columnname, startyear, endyear):
    lista = []
    for s in species:
        a = dif.loc[(dif['species'] == s) & (dif['year'] == startyear)]
        b = dif.loc[(dif['species'] == s) & (dif['year'] == endyear)]
        c = b - a
        lista.append({s: c})
        
    return lista 

# buscar los valores más altos. ABS!!! 

In [ ]:
# nombrecolumna = list(dif.columns)
species = list(dif.species.unique()) 
def compare(columnname, startyear, endyear):
    lista = []
    for s in species:
        a = dif.loc[(dif['species'] == s) & (dif['year'] == startyear)][columnname]
        b = dif.loc[(dif['species'] == s) & (dif['year'] == endyear)][columnname]
        c = b - a
        lista.append({s: c})
        
    return lista 

# buscar los valores más altos. ABS!!! 

In [25]:
# nombrecolumna = list(dif.columns)
species = list(dif.species.unique()) 
def compare(columnname, startyear, endyear):
    lista = {}
    for s in species:
        a = dif.loc[s, [(dif['species'] == s) & (dif['year'] == startyear)]]
        b = dif.loc[s, [(dif['species'] == s) & (dif['year'] == endyear)]]
        c = float(b.columnname) - float(a.columnname)
        lista.append({s: c})
        
    return lista

# buscar los valores más altos. ABS!!! 

In [26]:
compare(columnname = dif.lat_mean, 
        startyear = 2001, 
        endyear = 2020)

KeyError: 'Accipiter gentilis'

In [ ]:
d = {} 
a = dif.loc[(dif['species'] == 'Uria aalge') & (dif['year'] == 2020)]
b = dif.loc[(dif['species'] == 'Uria aalge') & (dif['year'] == 2001)]
c = float(a.lat_mean) - float(b.lat_mean)
d.append({'x': c})
d

In [ ]:
c = float(a.lat_mean) - float(b.lat_mean)
c

In [ ]:
for y in year_list:
    data = pd.DataFrame(y.groupby(['species'])['lat'].mean())

    
#buscar el modo de incluirlas en el dataframe cambiando el nombre
# hacer lo mismo para long
#for i, df in enumerate(year_list, 1):
 #   df.columns = [col_name+'_df{}'.format(i) for col_name in df.columns]
    

In [ ]:
# revisar

def plot_df(df, x, y, title="", xlabel='Date', ylabel='Value', dpi=100):
    plt.figure(figsize=(16,5), dpi = dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

plot_df(df = gbif_r, 
        x = gbif_r.date, 
        y = gbif_r.observations, 
        title = 'Bird ')    

In [ ]:
#  Usar Poisson para describir el número de ocurrencias en un determinado espacio de tiempo

In [ ]:
# nombrecolumna = list(dif.columns)

dif['mean_dif'] = dif.
species = list(dif.species.unique()) 
def compare(columnname, startyear, endyear):
    lista = {}
    for s in species:
        a = dif.loc[[(dif['species'] == s) & (dif['year'] == startyear)]]
        b = dif.loc[[(dif['species'] == s) & (dif['year'] == endyear)]]
        c = float(b.columnname) - float(a.columnname)
        lista.append({s: c})
        
    return lista

# buscar los valores más altos. ABS!!! 

In [22]:
# Checking changes in the geographic distribution of species
for s in dif.species:
    mobility = []
    if len(dif.regions.unique()) > 12:
        mobility.append(s)
    else:
        pass
    print(mobility)

['Xenus cinereus']


In [24]:
len(dif.regions.unique())

19

In [14]:
for s in dif.species:
    mobility = []
    if len(s) > 1:
        mobility.append(s)
    print(mobility)

['Accipiter gentilis']
['Accipiter nisus']
['Acrocephalus arundinaceus']
['Acrocephalus melanopogon']
['Acrocephalus paludicola']
['Acrocephalus schoenobaenus']
['Acrocephalus scirpaceus']
['Actitis hypoleucos']
['Aegithalos caudatus']
['Aegolius funereus']
['Aegypius monachus']
['Aix galericulata']
['Aix sponsa']
['Alauda arvensis']
['Alca torda']
['Alcedo atthis']
['Alectoris barbara']
['Alectoris rufa']
['Alle alle']
['Alopochen aegyptiaca']
['Amandava amandava']
['Anas acuta']
['Anas americana']
['Anas clypeata']
['Anas crecca']
['Anas penelope']
['Anas platyrhynchos']
['Anas querquedula']
['Anas strepera']
['Anser albifrons']
['Anser anser']
['Anser brachyrhynchus']
['Anser erythropus']
['Anser fabalis']
['Anser indicus']
['Anser serrirostris']
['Anthus berthelotii']
['Anthus campestris']
['Anthus hodgsoni']
['Anthus petrosus']
['Anthus pratensis']
['Anthus richardi']
['Anthus spinoletta']
['Anthus trivialis']
['Apus apus']
['Apus caffer']
['Apus melba']
['Apus pallidus']
['Apus u

['Turdus iliacus']
['Turdus merula']
['Turdus philomelos']
['Turdus pilaris']
['Turdus torquatus']
['Turdus viscivorus']
['Tyto alba']
['Upupa epops']
['Uria aalge']
['Vanellus gregarius']
['Vanellus vanellus']
['Xema sabini']
['Xenus cinereus']
['Accipiter gentilis']
['Accipiter nisus']
['Acridotheres tristis']
['Acrocephalus arundinaceus']
['Acrocephalus melanopogon']
['Acrocephalus paludicola']
['Acrocephalus schoenobaenus']
['Acrocephalus scirpaceus']
['Actitis hypoleucos']
['Actitis macularius']
['Aegithalos caudatus']
['Aegolius funereus']
['Aegypius monachus']
['Aix galericulata']
['Aix sponsa']
['Alauda arvensis']
['Alca torda']
['Alcedo atthis']
['Alectoris barbara']
['Alectoris rufa']
['Alle alle']
['Amandava amandava']
['Amazona ochrocephala']
['Anas acuta']
['Anas americana']
['Anas clypeata']
['Anas crecca']
['Anas discors']
['Anas penelope']
['Anas platyrhynchos']
['Anas querquedula']
['Anas rubripes']
['Anas strepera']
['Anser albifrons']
['Anser anser']
['Anser brachyrh

['Lanius senator']
['Larus argentatus']
['Larus cachinnans']
['Larus canus']
['Larus delawarensis']
['Larus fuscus']
['Larus glaucoides']
['Larus hyperboreus']
['Larus marinus']
['Larus michahellis']
['Leiothrix lutea']
['Leptoptilos crumenifer']
['Leucophaeus atricilla']
['Limicola falcinellus']
['Limosa lapponica']
['Limosa limosa']
['Linaria cannabina']
['Locustella luscinioides']
['Locustella naevia']
['Lophophanes cristatus']
['Loxia curvirostra']
['Lullula arborea']
['Luscinia megarhynchos']
['Luscinia svecica']
['Lymnocryptes minimus']
['Marmaronetta angustirostris']
['Melanitta fusca']
['Melanitta nigra']
['Melanitta perspicillata']
['Melanocorypha calandra']
['Mergellus albellus']
['Mergus serrator']
['Merops apiaster']
['Milvus migrans']
['Milvus milvus']
['Monticola saxatilis']
['Monticola solitarius']
['Montifringilla nivalis']
['Morus bassanus']
['Motacilla alba']
['Motacilla cinerea']
['Motacilla flava']
['Muscicapa striata']
['Myiopsitta monachus']
['Nandayus nenday']
['

['Turdus pilaris']
['Turdus torquatus']
['Turdus viscivorus']
['Tyto alba']
['Upupa epops']
['Uria aalge']
['Vanellus gregarius']
['Vanellus vanellus']
['Xema sabini']
['Xenus cinereus']
['Acanthis flammea']
['Accipiter gentilis']
['Accipiter nisus']
['Acrocephalus arundinaceus']
['Acrocephalus baeticatus']
['Acrocephalus melanopogon']
['Acrocephalus paludicola']
['Acrocephalus schoenobaenus']
['Acrocephalus scirpaceus']
['Actitis hypoleucos']
['Actitis macularius']
['Aegithalos caudatus']
['Aegolius funereus']
['Aegypius monachus']
['Agapornis roseicollis']
['Aix galericulata']
['Aix sponsa']
['Alauda arvensis']
['Alca torda']
['Alcedo atthis']
['Alectoris barbara']
['Alectoris rufa']
['Alopochen aegyptiaca']
['Amandava amandava']
['Anas acuta']
['Anas americana']
['Anas clypeata']
['Anas crecca']
['Anas discors']
['Anas penelope']
['Anas platyrhynchos']
['Anas querquedula']
['Anas rubripes']
['Anas strepera']
['Anser albifrons']
['Anser anser']
['Anser brachyrhynchus']
['Anser erythr

['Calidris maritima']
['Calidris melanotos']
['Calidris minuta']
['Calidris temminckii']
['Calonectris diomedea']
['Caprimulgus europaeus']
['Caprimulgus ruficollis']
['Carduelis carduelis']
['Carduelis citrinella']
['Carpodacus erythrinus']
['Cecropis daurica']
['Certhia brachydactyla']
['Certhia familiaris']
['Cettia cetti']
['Charadrius alexandrinus']
['Charadrius dubius']
['Charadrius hiaticula']
['Charadrius morinellus']
['Charadrius semipalmatus']
['Charadrius vociferus']
['Chersophilus duponti']
['Chlamydotis undulata']
['Chlidonias hybrida']
['Chlidonias leucopterus']
['Chlidonias niger']
['Chloris chloris']
['Chroicocephalus cirrocephalus']
['Chroicocephalus genei']
['Chroicocephalus philadelphia']
['Chroicocephalus ridibundus']
['Ciconia ciconia']
['Ciconia nigra']
['Cinclus cinclus']
['Circaetus gallicus']
['Circus aeruginosus']
['Circus cyaneus']
['Circus macrourus']
['Circus pygargus']
['Cisticola juncidis']
['Clamator glandarius']
['Clangula hyemalis']
['Coccothraustes co

['Phoenicurus ochruros']
['Phoenicurus phoenicurus']
['Phylloscopus bonelli']
['Phylloscopus canariensis']
['Phylloscopus collybita']
['Phylloscopus fuscatus']
['Phylloscopus humei']
['Phylloscopus ibericus']
['Phylloscopus inornatus']
['Phylloscopus proregulus']
['Phylloscopus sibillatrix']
['Phylloscopus trochilus']
['Pica pica']
['Picus vaillantii']
['Picus viridis']
['Platalea leucorodia']
['Plectrophenax nivalis']
['Plegadis falcinellus']
['Ploceus melanocephalus']
['Pluvialis apricaria']
['Pluvialis dominica']
['Pluvialis fulva']
['Pluvialis squatarola']
['Podiceps auritus']
['Podiceps cristatus']
['Podiceps grisegena']
['Podiceps nigricollis']
['Podilymbus podiceps']
['Poecile palustris']
['Porphyrio alleni']
['Porphyrio porphyrio']
['Porzana parva']
['Porzana porzana']
['Porzana pusilla']
['Prunella collaris']
['Prunella modularis']
['Psittacula krameri']
['Pterocles alchata']
['Pterocles orientalis']
['Ptyonoprogne rupestris']
['Puffinus baroli']
['Puffinus gravis']
['Puffinus

['Morus bassanus']
['Motacilla alba']
['Motacilla cinerea']
['Motacilla citreola']
['Motacilla flava']
['Motacilla tschutschensis']
['Muscicapa striata']
['Myiopsitta monachus']
['Nandayus nenday']
['Neophron percnopterus']
['Netta rufina']
['Numenius arquata']
['Numenius phaeopus']
['Nycticorax nycticorax']
['Nymphicus hollandicus']
['Oceanites oceanicus']
['Oceanodroma castro']
['Oceanodroma leucorhoa']
['Oenanthe deserti']
['Oenanthe hispanica']
['Oenanthe leucura']
['Oenanthe oenanthe']
['Oriolus oriolus']
['Otis tarda']
['Otus scops']
['Oxyura leucocephala']
['Pandion haliaetus']
['Panurus biarmicus']
['Parabuteo unicinctus']
['Parus major']
['Passer domesticus']
['Passer hispaniolensis']
['Passer italiae']
['Passer montanus']
['Pastor roseus']
['Pelagodroma marina']
['Perdix perdix']
['Periparus ater']
['Pernis apivorus']
['Petronia petronia']
['Phaethon aethereus']
['Phalacrocorax aristotelis']
['Phalacrocorax carbo']
['Phalaropus fulicarius']
['Phalaropus lobatus']
['Phasianus 

In [ ]:
for ind in df.index:
     print(df['Name'][ind], df['Stream'][ind])